# Getting started with your database
### Database-coding using SQLAlchemy and another (!!) Jupyter Notebook
Start a [Vanilla Jupyterlab](ControlBoard.ipynb#Vanilla-Jupyter-Datascience-Notebook) instance, then **copy** the code snippets below. Don't use this AWK DataLab Controlboard directly ;-).
### Imports

In [1]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy_utils import database_exists, create_database
from urllib import parse
from dotenv import dotenv_values

import pandas as pd


### Connect to your database
SQLAlchemy let's you use the same syntax and logic for different databases. All you need to change is the connection piece. You'll receive an `Engine`-object from SQLAlchemy (the connection won't be established until you do something with it). [Check here](https://docs.sqlalchemy.org/en/13/core/connections.html) to get started.

#### The Passwords will be different for you
To get the passwords below, use the initial `helm upgrade --install ...` command. Or (more complex), grab the [passwords from the respective Kubernetes secret](https://kubernetes.io/docs/tasks/configmap-secret/managing-secret-using-kubectl/#decoding-secret), e.g. named `postgresql`.

#### PostgreSQL (state of the art)

In [2]:
# Path and filename to your env file where you stored your credentials to your
# PostgreSQL db (see controlboard)
postgresql_file = './postgres.env'
# Name of your database - this database does NOT exist yet (create it below with `create_database()`)
database = 'my-new-database'

# Connection details - you should not need to change this
dialect = 'postgresql'  # Could be almost any other DB technology
host = 'postgresql'  # Name of the Kubernetes service
port = 5432

###############################
# Get credentials from env file
config = dotenv_values(postgresql_file)
username = config['USERNAME']
password = config['PASSWORD']

# URL-encode password for characters like %, ä, ...
password = parse.quote_plus(password)

url = f'{dialect}://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(url)

#### MySQL (connect as root)

In [3]:
# Path and filename to your env file where you stored your credentials to your
# PostgreSQL db (see controlboard)
mysql_file = './mysql.env'
# Name of your database - this database does NOT exist yet (create it below with `create_database()`)
database = 'my-new-database'

###############################
# Get credentials from env file
config = dotenv_values(mysql_file)
username = 'root'  # Only root user can create a new database
password = config['ROOT_PASSWORD']

# Connection details according to docker-compose.yml - do not change this
dialect = 'mysql+mysqlconnector'  # Could be almost any other DB technology
host = 'mysql'
port = 3306
# URL-encode password for characters like %, ä, ...
password = parse.quote_plus(password)

url = f'{dialect}://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(url)

KeyError: 'ROOT_PASSWORD'

***
## Create a new database once (you start with an empty database)

Create a new database called `my-new-database` (or anything, really). This command should return the value `True`, which means you could also successfully connect to the db

In [4]:
if not database_exists(engine.url):
    create_database(engine.url)

print(f'Database "{database}" exists: {database_exists(engine.url)}')

Database "my-new-database" exists: True


## Load example data into the database
### Postgres
Download `northwind.sql` from [this link](https://github.com/pthom/northwind_psql/raw/master/northwind.sql) (shift-click, then `Save link as...`), taken from the famous [Northwind example database](https://github.com/pthom/northwind_psql). Move the file into your `work` directory mounted in Jupyter.

In [6]:
sql = open("/home/jovyan/work/northwind.sql").read()
with engine.begin() as connection:
    connection.execute(sql)

### MySQL
MySQL needs more work. Download the database schema `northwind.sql` and the actual data `northwind-data.sql` from [this Github Repo](https://github.com/dalers/mywind). Move the 2 files into your `work` directory mounted in Jupyter.

Read the SQL commands in sequence and feed the individual commands (separated by a `;` and a subsequent line-break) individually to MySQL:

In [ ]:
for filename in ("/home/jovyan/work/northwind.sql", "/home/jovyan/work/northwind-data.sql"):
    sql = open(filename).read()
    with engine.begin() as connection:
        for command in sql.split(';\n'):
            if not command.strip() or command.startswith('--'):
                # Empty or commented line - MySQL would throw an exception
                continue
            connection.execute(command)

***
## Explore the DB
Apart from the database itself and the table, you might need to specify a schema. In our case:

In [7]:
# Postgres - let's use the standard/default schema
schema = 'public'

In [ ]:
# MySQL - the load above created its own schema
schema = 'northwind'

List all tables in the current database. SQLAlchemy uses an object called `MetaData` to describe the database:

In [8]:
# Associate the metadata with our database (the engine-object)
meta = MetaData(bind=engine, schema=schema)
# Load the existing database metadata from the database into meta
meta.reflect()
# Print all tables
meta.tables.keys()

dict_keys(['public.us_states', 'public.customers', 'public.orders', 'public.employees', 'public.shippers', 'public.products', 'public.categories', 'public.suppliers', 'public.order_details', 'public.region', 'public.territories', 'public.employee_territories', 'public.customer_demographics', 'public.customer_customer_demo'])

Print all columns of all tables of the current database:

In [9]:
for table in meta.sorted_tables:
    for column in table.columns:
        print(f'{table.name}: {column.name}')

categories: category_id
categories: category_name
categories: description
categories: picture
customer_demographics: customer_type_id
customer_demographics: customer_desc
customers: customer_id
customers: company_name
customers: contact_name
customers: contact_title
customers: address
customers: city
customers: region
customers: postal_code
customers: country
customers: phone
customers: fax
employees: employee_id
employees: last_name
employees: first_name
employees: title
employees: title_of_courtesy
employees: birth_date
employees: hire_date
employees: address
employees: city
employees: region
employees: postal_code
employees: country
employees: home_phone
employees: extension
employees: photo
employees: notes
employees: reports_to
employees: photo_path
region: region_id
region: region_description
shippers: shipper_id
shippers: company_name
shippers: phone
suppliers: supplier_id
suppliers: company_name
suppliers: contact_name
suppliers: contact_title
suppliers: address
suppliers: city

## SQLAlchemy and Pandas Dataframes
SQLAlchemy plays nicely with Pandas. In general, you pass the `Engine`-object to Pandas as well as the schema - that's it.

To get you started, try this to **read** an entire DB table into a dataframe `df`:

In [12]:
table_name = 'customers'
df = pd.read_sql_table(
    table_name,
    con=engine,
    schema=schema,
    index_col='customer_id'  # column name to use as dataframe-index (optional)
)
df

,company_name,contact_name,contact_title,address,city,region,postal_code,country,phone,fax
customer_id,,,,,,,,,,
ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545
ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745
ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,None,05023,Mexico,(5) 555-3932,None
AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67
...,...,...,...,...,...,...,...,...,...,...
WARTH,Wartian Herkku,Pirkko Koskitalo,Accounting Manager,Torikatu 38,Oulu,None,90110,Finland,981-443655,981-443655
WELLI,Wellington Importadora,Paula Parente,Sales Manager,"Rua do Mercado, 12",Resende,SP,08737-363,Brazil,(14) 555-8122,None
WHITC,White Clover Markets,Karl Jablonski,Owner,305 - 14th Ave. S. Suite 3B,Seattle,WA,98128,USA,(206) 555-4112,(206) 555-4115


To **write** a dataframe `df` to a new table, do this:

In [11]:
table_name = 'customers_copy'

df.to_sql(
    table_name,
    con=engine,
    schema=schema,
    if_exists='fail',  # What to do with an existing table? Could also be `replace` or `append`
    index=True,  # Whether to write the dataframe index as an additional column. Won't be a primary key automatically!
)

91

## Create an entity diagram to understand the structure of the database
[SQLAlchemy_Schemadisplay](https://github.com/sqlalchemy/sqlalchemy/wiki/SchemaDisplay) allows you to quickly see the structure of a DB like this: ![example schema](https://raw.githubusercontent.com/wiki/sqlalchemy/sqlalchemy/UsageRecipes/SchemaDisplay/schema.png)

In [13]:
from sqlalchemy_schemadisplay import create_schema_graph

**Postgres only:** We need to do some cleanup as SQLAlchemy did not recognize all DB types: `SAWarning: Did not recognize type 'bpchar' of column 'customer_id'`. Every column needs to have set a type.

In [14]:
# Postgres only
# SQLAlchemy has issues with the following columns when using Postgres. They all seem to be strings
offending = ['territory_description', 'region_description', 'customer_id', 'customer_type_id']

from sqlalchemy.types import VARCHAR

for table in meta.sorted_tables:
    for column in table.columns:
        if column.name in offending:
            print(f'{table.name}: {column.name}')
            column.type = VARCHAR(32)

customer_demographics: customer_type_id
customers: customer_id
region: region_description
customer_customer_demo: customer_id
customer_customer_demo: customer_type_id
orders: customer_id
territories: territory_description


Create the entity diagram. It will be saved as `db_entity_diagram.png`

In [15]:
graph = create_schema_graph(metadata=meta,
   show_datatypes=True,
   show_indexes=True,
   rankdir='LR', # From left to right (instead of top to bottom)
   concentrate=False # Don't try to join the relation lines together
)

graph.write_png('db_entity_diagram.png')